In [1234]:
import os

import datetime
import time

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

from scipy.stats import poisson

import matplotlib.pyplot as plt

In [1235]:
path='C:/Users/35266/Documents/Python Scripts/el/'
route='LUX-TXL'
risk_spill=0.8
risk_spoil=risk_spill
unique_identifier=['DepDate','FltNum','dtime']
cluster_variables=['dday','dtime','Direction','month']

# Determine downweight factor

In [1236]:
C_datadriven=pd.read_csv(path+'R_Training_Pax.csv')
TotalCap=pd.read_csv(path+'Capacity_forPlots_'+route+'.csv')
Prediction=pd.read_csv(path+'R_Output_Test_Pax.csv',sep=',')
GroupPax=pd.read_csv(path+'FrontEnd_Input/GroupPax_'+route+'.csv')

In [1237]:
C_datadriven=C_datadriven.groupby(list(set(unique_identifier+cluster_variables)))['NumPax'].sum().reset_index()

deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in C_datadriven['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([C_datadriven['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

C_datadriven=C_datadriven.merge(Map_DepTime,on='dtime')
C_datadriven['dtime']=C_datadriven['deptime']
C_datadriven=C_datadriven[[x for x in C_datadriven.columns if x!='deptime']]

C_datadriven=C_datadriven.merge(TotalCap.loc[TotalCap['Dprio']==1,:],on=unique_identifier)

C_datadriven['downweight']=C_datadriven['NumPax']/C_datadriven['Cabin Capacity']
C_datadriven=C_datadriven.groupby(cluster_variables)['downweight'].apply(lambda x: np.quantile(x,0.5)).reset_index()
C_datadriven['downweight']=C_datadriven['downweight'].apply(lambda x: 0.8 if x>0.8 else x)
C_datadriven['downweight']=C_datadriven['downweight'].apply(lambda x: 0.2 if x<0.2 else x)

# Import to total capacity

# Associate downweight factor and cabin capacity to prediction

In [1238]:
deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in Prediction['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([Prediction['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

Prediction=Prediction.merge(Map_DepTime,on='dtime')
Prediction['dtime']=Prediction['deptime']
Prediction=Prediction[[x for x in Prediction.columns if x!='deptime']]

Prediction=Prediction.merge(TotalCap[unique_identifier+['Dprio','Cabin Capacity']],
on=unique_identifier+['Dprio'])

Prediction=Prediction.merge(C_datadriven,on=cluster_variables,how='left')
Prediction['downweight']=Prediction['downweight'].apply(lambda x: 0.8 if pd.isnull(x) else x)

KeyError: 'dday'

In [ ]:
IdealCurve=Prediction[unique_identifier+['Dprio','forecast_bookings','Cabin Capacity','downweight']].copy()

In [ ]:
IdealCurve=IdealCurve.merge(IdealCurve.groupby(unique_identifier)['forecast_bookings'].sum().reset_index()\
.rename(columns={'forecast_bookings': 'forecast_bookings_sum'}),on=['DepDate','FltNum','dtime'])

In [ ]:
IdealCurve['forecast_bookings']=IdealCurve['forecast_bookings']/IdealCurve['forecast_bookings_sum']

In [ ]:
IdealCurve=IdealCurve[[x for x in IdealCurve.columns if x!='forecast_bookings_sum']]

In [ ]:
IdealCurve['lambda_100']=IdealCurve['Cabin Capacity']*IdealCurve['forecast_bookings']
IdealCurve['lambda_80']=IdealCurve['Cabin Capacity']*IdealCurve['downweight']*IdealCurve['forecast_bookings']

In [ ]:
IdealCurve=IdealCurve[unique_identifier+['Dprio']+[x for x in IdealCurve.columns if 'lambda' in x]]

In [ ]:
IdealCurve['Dprio']=-IdealCurve['Dprio']
IdealCurve=IdealCurve.set_index(['Dprio']+unique_identifier)

IdealCurve_cumul_100=IdealCurve.groupby(level=[x for x in range(len(unique_identifier)+1)]).sum()\
.groupby(level=[x for x in range(1,len(unique_identifier)+1)])['lambda_100'].cumsum().reset_index().rename(columns={'lambda_100': 'lambda_100_cumsum'})
IdealCurve_cumul_100['Dprio']=-IdealCurve_cumul_100['Dprio']

IdealCurve_cumul_80=IdealCurve.groupby(level=[x for x in range(len(unique_identifier)+1)]).sum()\
.groupby(level=[x for x in range(1,len(unique_identifier)+1)])['lambda_80'].cumsum().reset_index().rename(columns={'lambda_80': 'lambda_80_cumsum'})
IdealCurve_cumul_80['Dprio']=-IdealCurve_cumul_80['Dprio']

IdealCurve=IdealCurve.reset_index()
IdealCurve['Dprio']=-IdealCurve['Dprio']

IdealCurve=IdealCurve.set_index(['Dprio']+unique_identifier)

IdealCurve_cond_100=IdealCurve.groupby(level=[x for x in range(len(unique_identifier)+1)]).sum()\
.groupby(level=[x for x in range(1,len(unique_identifier)+1)])['lambda_100'].cumsum().reset_index().rename(columns={'lambda_100': 'lambda_100_cond'})

IdealCurve_cond_80=IdealCurve.groupby(level=[x for x in range(len(unique_identifier)+1)]).sum()\
.groupby(level=[x for x in range(1,len(unique_identifier)+1)])['lambda_80'].cumsum().reset_index().rename(columns={'lambda_80': 'lambda_80_cond'})

IdealCurve=IdealCurve.reset_index()

IdealCurve_cumul_100=IdealCurve_cumul_100.sort_values(by=['Dprio']+unique_identifier)
IdealCurve_cumul_80=IdealCurve_cumul_80.sort_values(by=['Dprio']+unique_identifier)
IdealCurve_cond_100=IdealCurve_cond_100.sort_values(by=['Dprio']+unique_identifier)
IdealCurve_cond_100['thres_100']=poisson.ppf(1-risk_spoil,IdealCurve_cond_100['lambda_100_cond'])
IdealCurve_cond_80=IdealCurve_cond_80.sort_values(by=['Dprio']+unique_identifier)
IdealCurve_cond_80['thres_80']=poisson.ppf(risk_spoil,IdealCurve_cond_80['lambda_80_cond'])

In [ ]:
IdealCurve_cond_100=IdealCurve_cond_100\
.merge(Prediction[['Dprio']+unique_identifier+['Cabin Capacity']],on=['Dprio']+unique_identifier)

IdealCurve_cond_80=IdealCurve_cond_80\
.merge(Prediction[['Dprio']+unique_identifier+['Cabin Capacity','downweight']],on=['Dprio']+unique_identifier)

In [ ]:
IdealCurve_cond_100['Ideal_upper']=IdealCurve_cond_100['Cabin Capacity']-IdealCurve_cond_100['thres_100']
IdealCurve_cond_100['Ideal_upper']=IdealCurve_cond_100[['Ideal_upper','Cabin Capacity']].apply(lambda x: x[1] if x[0]>x[1] else x[0],axis=1)
IdealCurve_cond_80['Ideal_lower']=IdealCurve_cond_80['Cabin Capacity']*IdealCurve_cond_80['downweight']-IdealCurve_cond_80['thres_80']
IdealCurve_cond_80['Ideal_lower']=IdealCurve_cond_80['Ideal_lower'].apply(lambda x: 0 if x<0 else x)

In [ ]:
IdealCurve=IdealCurve_cumul_100.merge(IdealCurve_cumul_80,on=['Dprio']+unique_identifier)\
.merge(IdealCurve_cond_80[['Dprio']+unique_identifier+['Ideal_lower']],on=['Dprio']+unique_identifier)\
.merge(IdealCurve_cond_100[['Dprio']+unique_identifier+['Ideal_upper']],on=['Dprio']+unique_identifier)

In [ ]:
IdealCurve=IdealCurve[['Dprio']+unique_identifier+['Ideal_lower','lambda_80_cumsum','lambda_100_cumsum','Ideal_upper']]

In [ ]:
IdealCurve.columns=['Dprio']+unique_identifier+['Ramp-up frontier','Ideal curve (80% LF)','Ideal curve (100% LF)','Phase-down frontier']

### Remove impact of group pax

In [ ]:
IdealCurve=IdealCurve.merge(GroupPax[['Dprio']+unique_identifier+['Group_pax_cumul']],on=['Dprio']+unique_identifier)

IdealCurve=IdealCurve.merge(TotalCap[['Dprio']+unique_identifier+['Cabin Capacity']],on=['Dprio']+unique_identifier)

IdealCurve=IdealCurve.merge(IdealCurve.loc[IdealCurve['Dprio']==1,unique_identifier+['Ideal curve (80% LF)','Ideal curve (100% LF)']]\
.rename(columns={'Ideal curve (100% LF)': 'C_100', 'Ideal curve (80% LF)': 'C_80'}),on=unique_identifier)

In [ ]:
IdealCurve['Ideal curve (100% LF)']=IdealCurve['Ideal curve (100% LF)']\
*((IdealCurve['Cabin Capacity']-IdealCurve['Group_pax_cumul'])/IdealCurve['Cabin Capacity'])
IdealCurve['Phase-down frontier']=IdealCurve['Phase-down frontier']\
*((IdealCurve['Cabin Capacity']-IdealCurve['Group_pax_cumul'])/IdealCurve['Cabin Capacity'])
# IdealCurve['Ideal curve (80% LF)']=IdealCurve[['Ideal curve (80% LF)','Group_pax_cumul','C_100','C_80']]\
# .apply(lambda x: x[0] if x[2]-x[1]>x[3] else x[0]-(x[3]-(x[2]-x[1])),axis=1)
# IdealCurve['Ramp-up frontier']=IdealCurve[['Ramp-up frontier','Group_pax_cumul','C_100','C_80']]\
# .apply(lambda x: x[0] if x[2]-x[1]>x[3] else x[0]-(x[3]-(x[2]-x[1])),axis=1)
IdealCurve['Ideal curve (80% LF)']=IdealCurve['Ideal curve (80% LF)']\
*((IdealCurve['Cabin Capacity']-IdealCurve['Group_pax_cumul'])/IdealCurve['Cabin Capacity'])
IdealCurve['Ramp-up frontier']=IdealCurve['Ramp-up frontier']\
*((IdealCurve['Cabin Capacity']-IdealCurve['Group_pax_cumul'])/IdealCurve['Cabin Capacity'])

In [ ]:
IdealCurve['Ideal curve (100% LF)']=IdealCurve['Ideal curve (100% LF)'].apply(lambda x: 0 if x<0 else x)
IdealCurve['Phase-down frontier']=IdealCurve['Phase-down frontier'].apply(lambda x: 0 if x<0 else x)
IdealCurve['Ideal curve (80% LF)']=IdealCurve['Ideal curve (80% LF)'].apply(lambda x: 0 if x<0 else x)
IdealCurve['Ramp-up frontier']=IdealCurve['Ramp-up frontier'].apply(lambda x: 0 if x<0 else x)

In [ ]:
IdealCurve=IdealCurve[[x for x in IdealCurve.columns if 'cumul' not in x and 'C_' not in x and 'Capacity' not in x]]

# Risk and Actual Bookings

In [ ]:
Risk=Prediction[['Dprio']+unique_identifier+['NumPax']].copy()

In [ ]:
Mapping_StrToDt=pd.concat([Series(Risk['DepDate'].unique()),
Series([pd.to_datetime(x) for x in Risk['DepDate'].unique()])],axis=1)
Mapping_StrToDt.columns=['DepDate','DepDate_new']
Risk=Risk.merge(Mapping_StrToDt,on='DepDate')
Risk['DepDate']=Risk['DepDate_new']
Risk=Risk[[x for x in Risk.columns if x!='DepDate_new']]

In [ ]:
Risk['Dprio']=-Risk['Dprio']
Risk=Risk.set_index(['Dprio']+unique_identifier)

Risk=Risk.groupby(level=[x for x in range(len(unique_identifier)+1)]).sum()\
.groupby(level=[x for x in range(1,len(unique_identifier)+1)])['NumPax'].cumsum().reset_index()\
.rename(columns={'NumPax': 'NumPax_cumsum'})

Risk['Dprio']=-Risk['Dprio']

In [ ]:
Risk['IssueDate']=Risk['DepDate']-Risk['Dprio'].apply(lambda x: datetime.timedelta(x-1))

### Intermezzo for creating Actuals

In [ ]:
Actuals=Risk.loc[Risk['IssueDate']<=pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d')),:]
Actuals=Actuals[[x for x in Actuals.columns if x!='IssueDate']]
Actuals=Actuals.rename(columns={'NumPax_cumsum': 'Actual Bookings'})

### Continue with Risk

In [ ]:
Risk=Risk.loc[Risk['IssueDate']==pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d')),:]

In [ ]:
Risk=Risk[[x for x in Risk.columns if x!='IssueDate']]

In [ ]:
Risk['DepDate']=Risk['DepDate'].apply(lambda x: str(x).split(' ')[0])

In [ ]:
Risk=Risk.merge(IdealCurve.loc[IdealCurve['Dprio']==1,unique_identifier+['Ideal curve (80% LF)','Ideal curve (100% LF)']]\
.rename(columns={'Ideal curve (100% LF)': 'C_100', 'Ideal curve (80% LF)': 'C_80'}),on=unique_identifier)

In [ ]:
Risk=Risk.merge(IdealCurve_cond_100[['Dprio']+unique_identifier+['lambda_100_cond']],on=['Dprio']+unique_identifier)\
.merge(IdealCurve_cond_80[['Dprio']+unique_identifier+['lambda_80_cond']],on=['Dprio']+unique_identifier)

In [ ]:
Risk['SpillageRisk']=1-poisson.cdf(Risk['C_100']-Risk['NumPax_cumsum'],Risk['lambda_100_cond'])
Risk['SpoilageRisk']=poisson.cdf(Risk['C_80']-Risk['NumPax_cumsum'],Risk['lambda_80_cond'])

In [ ]:
Risk=Risk.merge(IdealCurve[['Dprio']+unique_identifier+['Ramp-up frontier','Phase-down frontier']],
          on=['Dprio']+unique_identifier)

In [ ]:
Risk['Intensity_downweighted']=Risk['NumPax_cumsum']-Risk['Ramp-up frontier']
Risk['Intensity_full']=Risk['NumPax_cumsum']-Risk['Phase-down frontier']

In [ ]:
Risk=Risk[['DepDate','FltNum','dtime','SpoilageRisk','Intensity_downweighted','Intensity_full','SpillageRisk']]

# Capacity

In [ ]:
TotalCap=TotalCap.merge(GroupPax[['Dprio']+unique_identifier+['Group_pax_cumul']],on=['Dprio']+unique_identifier)

TotalCap['Real Cabin Capacity']=TotalCap['Cabin Capacity']-TotalCap['Group_pax_cumul']
TotalCap=TotalCap.rename(columns={'Cabin Capacity': 'Initial Cabin Capacity'})

TotalCap.to_csv(path+'FrontEnd_Input/Capacity_'+route+'.csv',index=False)

# Data Export

In [ ]:
Risk=Risk.merge(Prediction[unique_identifier+['Direction','month','dday']].drop_duplicates(subset=unique_identifier),on=unique_identifier)
Risk['Route']=route

MonthMapping=DataFrame([[x for x in range(1,13)],['January','February','March','April','May','June','July','August',
                'September','October','November','December']]).transpose()
MonthMapping.columns=['month','Month']
MonthMapping['month']=MonthMapping['month'].astype('int') 
Risk=Risk.merge(MonthMapping,on='month')
Risk['month']=Risk['Month']
Risk=Risk[[x for x in Risk.columns if x!='Month']]

Risk=Risk[['Route','DepDate', 'FltNum', 'dtime', 'Direction', 'month', 'dday',
       'SpoilageRisk', 'Intensity_downweighted', 'Intensity_full',
       'SpillageRisk']]

Risk=Risk.sort_values(by=['DepDate', 'FltNum', 'dtime', 'Direction'])

Risk.to_csv(path+'FrontEnd_Input/Risk_'+route+'.csv',index=False)

In [ ]:
Actuals['DepDate']=Actuals['DepDate'].apply(lambda x: str(x).split(' ')[0])

Actuals=Actuals.merge(Prediction[unique_identifier+['Direction','month','dday']].drop_duplicates(subset=unique_identifier),
on=unique_identifier)
Actuals['Route']=route

Actuals=Actuals[['Route','DepDate', 'FltNum', 'dtime', 'Direction', 'month', 'dday', 'Dprio','Actual Bookings']]

Actuals['Actual Bookings']=Actuals['Actual Bookings'].apply(lambda x: x if x>=0 else 0)

Actuals.to_csv(path+'FrontEnd_Input/Actuals_'+route+'.csv',index=False)

In [ ]:
IdealCurve=IdealCurve.merge(Prediction[unique_identifier+['Direction','month','dday']].drop_duplicates(subset=unique_identifier),
on=unique_identifier)

IdealCurve['Route']=route

IdealCurve=IdealCurve[['Route','DepDate', 'FltNum', 'dtime', 'Direction', 'month', 'dday', 'Dprio',
'Ramp-up frontier','Ideal curve (80% LF)','Ideal curve (100% LF)','Phase-down frontier']]

IdealCurve.to_csv(path+'FrontEnd_Input/IdealCurve_'+route+'.csv',index=False)